In [24]:
import numpy as np
import os

In [25]:
# import sys
# print(sys.path)

# Part 1

In [26]:
ES_TRAINING_DATA = "./ES/train"
ES_TESTING_DATA = "./ES/dev.in"
ES_LABELED_DATA = "./ES/dev.out"

RU_TRAINING_DATA = "./RU/train"
RU_TESTING_DATA = "./RU/dev.in"
RU_LABELED_DATA = "./RU/dev.out"

def readTrainingData(filePath):
    data = []
    with open(filePath) as file:
        data = file.read().splitlines()
    
    output = []
    for line in data:
        line = line.split(" ")
        if(len(line) > 2):
            # . … O
            line = [" ".join(line[0:len(line)-1]), line[len(line)-1]]
            output.append(line)
        else:
            output.append(line)
        
    return output


def readTestData(filePath):
    data = []
    with open(filePath) as file:
        data = file.read().splitlines()
    
    output = []
    for line in data:
        output.append(line)
    
    return output

es_training_data = readTrainingData(ES_TRAINING_DATA)
ru_training_data = readTrainingData(RU_TRAINING_DATA)
es_testing_data = readTestData(ES_TESTING_DATA)
ru_testing_data = readTestData(RU_TESTING_DATA)
es_labeled_data = readTrainingData(ES_LABELED_DATA)
ru_labeled_data = readTrainingData(RU_LABELED_DATA)

In [27]:
"""
Write a function that estimates the emission parameters from the training set using MLE (maximum
likelihood estimation)

One problem with estimating the emission parameters is that some words that appear in the test set
do not appear in the training set. One simple idea to handle this issue is as follows. We introduce
a special word token #UNK#, and make the following modifications to the computation of emission
probabilitie
"""

def emissionMLE(dataset, k=1):
    observations = {} 
    tags = {} # tag/state
    tagToObs = {}

    for data in dataset:
        if len(data) < 2:
            continue
        o = data[0]
        t = data[1]

        if o not in observations:
            observations[o] = 0
        observations[o] += 1

        if t not in tags:
            tags[t] = 0
        tags[t] += 1

        key = (t, o)
        if key not in tagToObs:
            tagToObs[key] = 0
        tagToObs[key] += 1

    res = {}
    for key in tagToObs.keys():
        res[key] = tagToObs[key] / (tags[key[0]] + k)
    for key in tags.keys():
        unk_key = (key, "#UNK#")
        res[unk_key] = k / (tags[key] + k)
    
    return res

es_emission = emissionMLE(dataset=es_training_data, k=1)
ru_emission = emissionMLE(dataset=ru_training_data, k=1)


In [28]:
"""
Implement a simple sentiment analysis system that produces the tag for each word x in the sequence
"""

def argmax(emission):
    map = {}
    res = {}
    for key, value in emission.items():
        t = key[0]
        o = key[1]

        if o not in map:
            map[o] = value
            res[o] = t
        else:
            if value > map[o]:
                map[o] = value
                res[o] = t

    return res

es_seqmap = argmax(es_emission)
ru_seqmap = argmax(ru_emission)

ES_PRED = "./ES/dev.p1.out"
RU_PRED = "./RU/dev.p1.out"

def serializeSeqmapAndWriteToFile(seqmap, testData, predFilePath):
    serializedSeqmap = []
    with open(predFilePath, "w") as file:
        for data in testData:
            if data is None or len(data) == 0:
                file.write("\n")
                continue
            entry = [data]
            if data not in seqmap:
                entry.append("#UNK#")
            else:
                entry.append(seqmap[data])
            serializedSeqmap.append(entry)
            file.write("{} {}\n".format(entry[0], entry[1]))

    return serializedSeqmap
        
es_serialized_seqmap = serializeSeqmapAndWriteToFile(es_seqmap, es_testing_data, ES_PRED)
ru_serialized_seqmap = serializeSeqmapAndWriteToFile(ru_seqmap, ru_testing_data, RU_PRED)

# print(len(es_testing_data))
# print(len(es_testing_data[0]))
# print(len(es_seialized_seqmap))
# print(len(es_seialized_seqmap[0]))
# print(len(es_labeled_data))
# print(len(es_labeled_data[0]))


def calculateScore(LABELED_DATA, PREDICTED_DATA):
    os.system("python3 EvalScript/evalResult.py {} {}".format(LABELED_DATA, PREDICTED_DATA))


calculateScore(ES_LABELED_DATA, ES_PRED)
calculateScore(RU_LABELED_DATA, RU_PRED)


#Entity in gold data: 255
#Entity in prediction: 1288

#Correct Entity : 184
Entity  precision: 0.1429
Entity  recall: 0.7216
Entity  F: 0.2385

#Correct Sentiment : 109
Sentiment  precision: 0.0846
Sentiment  recall: 0.4275
Sentiment  F: 0.1413

#Entity in gold data: 461
#Entity in prediction: 1377

#Correct Entity : 301
Entity  precision: 0.2186
Entity  recall: 0.6529
Entity  F: 0.3275

#Correct Sentiment : 131
Sentiment  precision: 0.0951
Sentiment  recall: 0.2842
Sentiment  F: 0.1425
